In [6]:
import cv2
from collections import deque
import os
import shutil
import threading
import queue
from concurrent.futures import ThreadPoolExecutor
import time
from dataclasses import dataclass
from typing import Dict, Optional
import numpy as np

from ultralytics import YOLO

@dataclass
class VideoWriterState:
    writer: cv2.VideoWriter
    frame_queue: queue.Queue
    active: bool = True

class ThreadedVideoWriter:
    def __init__(self, path: str, fourcc: int, fps: float, frame_size: tuple):
        self.writer = cv2.VideoWriter(path, fourcc, fps, frame_size)
        self.frame_queue = queue.Queue(maxsize=300)  # Buffer size of 300 frames
        self.active = True
        self.thread = threading.Thread(target=self._write_frames)
        self.thread.daemon = True
        self.thread.start()
        self.frames_written = 0

    def _write_frames(self):
        while self.active or not self.frame_queue.empty():
            try:
                frame = self.frame_queue.get(timeout=1.0)
                self.writer.write(frame)
                self.frames_written += 1
                self.frame_queue.task_done()
            except queue.Empty:
                continue
            except Exception as e:
                print(f"Error writing frame: {e}")

    def write(self, frame):
        if self.active:
            try:
                self.frame_queue.put(frame.copy(), timeout=1.0)
            except queue.Full:
                print("[WARNING] Frame queue full, dropping frame")

    def release(self):
        self.active = False
        self.thread.join()
        self.writer.release()
        return self.frames_written > 0  # Return True if any frames were written

def live_yolo_video_splitter(
    output_dir="output_videos",
    yolo_model_path="yolov8n.pt",
    confidence_threshold=0.5,
    buffer_size=60,
    max_frames_no_person_after=60,
    fps=30,
    consecutive_frames_needed=5
):
    """
    Live monitoring of webcam to split into videos when a person is detected.
    Uses multithreading for video writing to prevent frame drops.
    """
    # Create directory structure
    os.makedirs(output_dir, exist_ok=True)
    before_dir = os.path.join(output_dir, "before_no_person")
    after_dir = os.path.join(output_dir, "after_no_person")
    person_dir = os.path.join(output_dir, "person")
    no_person_dir = os.path.join(output_dir, "no_person")
    temp_dir = os.path.join(output_dir, "temp")
    
    for dir_path in [before_dir, after_dir, person_dir, no_person_dir, temp_dir]:
        os.makedirs(dir_path, exist_ok=True)

    # Initialize YOLO
    model = YOLO(yolo_model_path)

    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    # Get frame dimensions
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (width, height)

    # Test and set video codec
    try:
        fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H264 codec
        test_path = os.path.join(temp_dir, 'test.mp4')
        test_writer = cv2.VideoWriter(test_path, fourcc, fps, frame_size)
        test_writer.release()
        os.remove(test_path)
    except:
        print("[WARNING] H264 codec not available, falling back to mp4v")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Initialize state
    buffer_frames = deque(maxlen=buffer_size)
    person_in_frame = False
    consecutive_person_frames = 0
    consecutive_noperson_frames = 0
    video_id = 1
    no_person_after_count = 0
    writers: Dict[str, Optional[ThreadedVideoWriter]] = {
        'before': None,
        'person': None,
        'after': None,
        'no_person': None
    }
    temp_files = {
        'before': None,
        'person': None,
        'after': None,
        'no_person': None
    }

    def get_temp_paths(vid_id):
        return {
            'before': os.path.join(temp_dir, f"before_no_person_{vid_id}.mp4"),
            'person': os.path.join(temp_dir, f"person_{vid_id}.mp4"),
            'after': os.path.join(temp_dir, f"after_no_person_{vid_id}.mp4"),
            'no_person': os.path.join(temp_dir, f"no_person_{vid_id}.mp4")
        }

    def get_final_paths(vid_id):
        return {
            'before': os.path.join(before_dir, f"before_no_person_{vid_id}.mp4"),
            'person': os.path.join(person_dir, f"person_{vid_id}.mp4"),
            'after': os.path.join(after_dir, f"after_no_person_{vid_id}.mp4"),
            'no_person': os.path.join(no_person_dir, f"no_person_{vid_id}.mp4")
        }

    def close_and_move_recordings(current_video_id):
        nonlocal writers, temp_files
        
        # Close all writers and wait for threads to finish
        valid_files = {}
        for key, writer in writers.items():
            if writer is not None:
                frames_written = writer.release()
                if frames_written:
                    valid_files[key] = temp_files[key]

        # Move valid files to final location
        final_paths = get_final_paths(current_video_id)
        for key, temp_path in valid_files.items():
            if os.path.exists(temp_path):
                try:
                    # Verify file is valid
                    cap = cv2.VideoCapture(temp_path)
                    if cap.isOpened():
                        cap.release()
                        shutil.move(temp_path, final_paths[key])
                        print(f"[INFO] Moved {os.path.basename(temp_path)} to final location")
                    else:
                        print(f"[WARNING] Corrupted file detected: {temp_path}")
                        os.remove(temp_path)
                except Exception as e:
                    print(f"[ERROR] Failed to move {temp_path}: {e}")

        writers = {key: None for key in writers}
        temp_files = {key: None for key in temp_files}

    def start_new_recordings():
        nonlocal writers, temp_files, video_id
        
        # Close and move previous recordings
        if any(writers.values()):
            close_and_move_recordings(video_id - 1)
            time.sleep(0.1)  # Small delay to ensure files are closed

        # Initialize new temp paths
        temp_files = get_temp_paths(video_id)
        
        # Create new writers
        writers = {
            'before': ThreadedVideoWriter(temp_files['before'], fourcc, fps, frame_size),
            'person': ThreadedVideoWriter(temp_files['person'], fourcc, fps, frame_size),
            'after': ThreadedVideoWriter(temp_files['after'], fourcc, fps, frame_size),
            'no_person': ThreadedVideoWriter(temp_files['no_person'], fourcc, fps, frame_size)
        }

        print(f"[INFO] Starting new recordings (ID: {video_id})")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("[INFO] Cannot read from camera or end of stream.")
                break

            # Store a copy of the frame in the buffer
            buffer_frames.append(frame.copy())
            
            # Process frame with YOLO
            results = model(frame, conf=confidence_threshold)
            has_person = any(int(box.cls[0]) == 0 for box in results[0].boxes)

            if has_person:
                consecutive_person_frames += 1
                consecutive_noperson_frames = 0
            else:
                consecutive_noperson_frames += 1
                consecutive_person_frames = 0

            # Handle transition to person detected
            if not person_in_frame and consecutive_person_frames >= consecutive_frames_needed:
                person_in_frame = True
                no_person_after_count = 0
                start_new_recordings()
                
                # Write buffer to before_no_person and no_person
                if writers['before'] and writers['no_person']:
                    for bf in buffer_frames:
                        writers['before'].write(bf)
                        writers['no_person'].write(bf)

            # Handle transition to no person
            elif person_in_frame and consecutive_noperson_frames >= consecutive_frames_needed:
                person_in_frame = False
                no_person_after_count = 0

            # Write current frame
            if person_in_frame:
                if writers['person']:
                    writers['person'].write(frame)
            else:
                if any(writers.values()):
                    no_person_after_count += 1
                    if writers['after']:
                        writers['after'].write(frame)
                    if writers['no_person']:
                        writers['no_person'].write(frame)
                    
                    if no_person_after_count >= max_frames_no_person_after:
                        close_and_move_recordings(video_id)
                        video_id += 1

            cv2.imshow("Live Feed", frame)
            if cv2.waitKey(1) & 0xFF == 27:
                break

    except KeyboardInterrupt:
        print("[INFO] Interrupted by user.")
    finally:
        # Cleanup
        if any(writers.values()):
            close_and_move_recordings(video_id)
        
        cap.release()
        cv2.destroyAllWindows()
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
        print("[INFO] Exiting...")

if __name__ == "__main__":
    live_yolo_video_splitter()


0: 480x640 1 bed, 23.4ms
Speed: 1.0ms preprocess, 23.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 1 bed, 21.0ms
Speed: 1.5ms preprocess, 21.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bed, 23.0ms
Speed: 1.0ms preprocess, 23.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bed, 22.1ms
Speed: 1.0ms preprocess, 22.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 1 bed, 24.4ms
Speed: 1.0ms preprocess, 24.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bed, 23.3ms
Speed: 0.9ms preprocess, 23.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bed, 24.2ms
Speed: 0.7ms preprocess, 24.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 23.0ms
Speed: 1.2ms preprocess, 23.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480

In [8]:
#slimmed down works well

import cv2
from collections import deque
import os
import shutil

# Import the YOLO class from ultralytics
from ultralytics import YOLO

def live_yolo_video_splitter(
    output_dir="output_videos",
    yolo_model_path="yolov8n.pt",
    confidence_threshold=0.5,
    buffer_size=60,
    max_frames_no_person_after=60,
    fps=30,
    consecutive_frames_needed=5
):
    """
    Live monitoring of webcam to split into two videos whenever a person is detected.
    Videos are written to a temporary directory and only moved to final location
    when the next recording starts.
    """
    # Create the output and temp directories
    os.makedirs(output_dir, exist_ok=True)
    temp_dir = os.path.join(output_dir, "temp")
    os.makedirs(temp_dir, exist_ok=True)

    # Initialize the YOLOv8 model
    model = YOLO(yolo_model_path)

    # Initialize webcam capture
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    # Get frame dimensions
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Use H264 codec if available, fallback to mp4v if not
    try:
        fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H264 codec
        test_writer = cv2.VideoWriter(
            os.path.join(temp_dir, 'test.mp4'),
            fourcc, fps, (width, height)
        )
        test_writer.release()
        os.remove(os.path.join(temp_dir, 'test.mp4'))
    except:
        print("[WARNING] H264 codec not available, falling back to mp4v")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Rolling buffer for the last `buffer_size` frames
    buffer_frames = deque(maxlen=buffer_size)

    # State variables
    person_in_frame = False
    consecutive_person_frames = 0
    consecutive_noperson_frames = 0

    # Video control
    video_id = 1
    out_no_person = None
    out_person = None
    recording_person = False
    recording_no_person = False
    no_person_after_count = 0
    previous_temp_files = []  # Track previous recording's files

    def start_new_recordings():
        nonlocal out_no_person, out_person, video_id, recording_person, recording_no_person, previous_temp_files
        
        # Move previous recordings to final location if they exist
        for temp_path in previous_temp_files:
            if os.path.exists(temp_path):
                final_path = os.path.join(output_dir, os.path.basename(temp_path))
                shutil.move(temp_path, final_path)
                print(f"[INFO] Moved {os.path.basename(temp_path)} to final location")
        
        # Start new recordings
        temp_no_person = os.path.join(temp_dir, f"no_person_{video_id}.mp4")
        temp_person = os.path.join(temp_dir, f"person_{video_id}.mp4")
        
        out_no_person = cv2.VideoWriter(temp_no_person, fourcc, fps, (width, height))
        out_person = cv2.VideoWriter(temp_person, fourcc, fps, (width, height))

        recording_no_person = True
        recording_person = True
        
        # Store current paths as previous for next cycle
        previous_temp_files = [temp_no_person, temp_person]

        print(f"[INFO] Starting new recordings (ID: {video_id})")

    def close_recordings():
        nonlocal out_no_person, out_person, recording_person, recording_no_person
        if out_no_person is not None:
            out_no_person.release()
            out_no_person = None
        if out_person is not None:
            out_person.release()
            out_person = None

        recording_person = False
        recording_no_person = False

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("[INFO] Cannot read from camera or end of stream.")
                break

            # Rest of the main loop remains exactly the same
            buffer_frames.append(frame)
            results = model(frame, conf=confidence_threshold)
            yolo_boxes = results[0].boxes
            has_person = any(int(box.cls[0]) == 0 for box in yolo_boxes)

            if has_person:
                consecutive_person_frames += 1
                consecutive_noperson_frames = 0
            else:
                consecutive_noperson_frames += 1
                consecutive_person_frames = 0

            if not person_in_frame:
                if consecutive_person_frames >= consecutive_frames_needed:
                    person_in_frame = True
                    no_person_after_count = 0
                    start_new_recordings()
                    if out_no_person:
                        for bf in buffer_frames:
                            out_no_person.write(bf)
            else:
                if consecutive_noperson_frames >= consecutive_frames_needed:
                    person_in_frame = False
                    no_person_after_count = 0

            if person_in_frame:
                if recording_person and out_person:
                    out_person.write(frame)
            else:
                if recording_person:
                    no_person_after_count += 1
                    if recording_no_person and out_no_person:
                        out_no_person.write(frame)

                    if no_person_after_count >= max_frames_no_person_after:
                        close_recordings()
                        video_id += 1

            cv2.imshow("Live Feed", frame)
            if cv2.waitKey(1) & 0xFF == 27:
                break

    except KeyboardInterrupt:
        print("[INFO] Interrupted by user.")
    finally:
        # Close current recordings
        close_recordings()
        
        # Move the last set of recordings to final location
        for temp_path in previous_temp_files:
            if os.path.exists(temp_path):
                final_path = os.path.join(output_dir, os.path.basename(temp_path))
                shutil.move(temp_path, final_path)
                print(f"[INFO] Moved final {os.path.basename(temp_path)} to final location")
        
        cap.release()
        cv2.destroyAllWindows()
        # Clean up temp directory
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
        print("[INFO] Exiting...")

if __name__ == "__main__":
    live_yolo_video_splitter()
    print('hello')


0: 480x640 1 bottle, 1 bed, 25.5ms
Speed: 0.6ms preprocess, 25.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 1 bed, 23.0ms
Speed: 1.0ms preprocess, 23.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 1 bed, 23.5ms
Speed: 1.0ms preprocess, 23.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 1 bed, 22.4ms
Speed: 1.0ms preprocess, 22.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 1 bed, 1 sink, 23.8ms
Speed: 1.0ms preprocess, 23.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 1 bed, 23.0ms
Speed: 1.2ms preprocess, 23.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 1 bed, 23.0ms
Speed: 1.1ms preprocess, 23.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bed, 23.6ms
Speed: 0.0ms preprocess, 23.6ms inf